# CRISP-DM Applied to Used Car Pricing

This notebook follows the CRISP-DM framework to analyze `vehicles.csv` and determine what factors influence used car prices. Deliverables include exploratory data analysis, data preparation, modeling, evaluation, and recommendations for a used car dealership.

**File used:** `/mnt/data/vehicles.csv`

## 1. Business Understanding

- **Objective:** Identify the factors that make a car more or less expensive and produce recommendations for inventory decisions at a used car dealership.
- **Success criteria:** A predictive model that reasonably explains price variation and a clear set of factors (feature importances / coefficients) that dealers can apply.


In [ ]:
import pandas as pd
from pathlib import Path

csv_path = Path('/mnt/data/vehicles.csv')
df = pd.read_csv(csv_path)
df.shape


In [ ]:
df.head()


In [ ]:
df.info()


In [ ]:
import numpy as np

missing = df.isnull().mean().sort_values(ascending=False)
missing[missing>0].head(20)


In [ ]:
df.describe(include='all').T


## 3. Data Preparation

- Remove rows with missing or zero price
- Create `age` feature from `year`
- Clean `odometer` and remove extreme outliers
- Keep a subset of relevant categorical features and reduce high-cardinality categories


In [ ]:
# Basic cleaning
import numpy as np

# copy
cars = df.copy()
# Ensure price column exists
if 'price' not in cars.columns:
    raise KeyError('price column not found in dataset')

# Drop rows with missing or non-positive price
cars = cars[cars['price'].notna()]
cars = cars[cars['price']>0]

# Year -> age (use 2025 as reference if year present; adjust if needed)
current_year = 2025
if 'year' in cars.columns:
    cars['age'] = current_year - cars['year']
    # remove unrealistic ages
    cars = cars[(cars['age']>=0) & (cars['age']<=120)]

# Clean odometer
if 'odometer' in cars.columns:
    cars = cars[cars['odometer'].notna()]
    cars = cars[cars['odometer']>=0]

# Keep relevant columns if present
candidate_cols = ['price','year','age','manufacturer','model','condition','cylinders','fuel','odometer',
                  'title_status','transmission','drive','size','type','paint_color','state']
existing_cols = [c for c in candidate_cols if c in cars.columns]
cars = cars[existing_cols]

print('shape after basic cleaning:', cars.shape)
cars.head()


In [ ]:
# Handle price distribution: log-transform and remove extreme outliers
import numpy as np

cars = cars.copy()
# remove top 0.5% and bottom 0.5% by price to reduce extreme influence
low_q, high_q = cars['price'].quantile(0.005), cars['price'].quantile(0.995)
cars = cars[(cars['price']>=low_q) & (cars['price']<=high_q)]

cars['log_price'] = np.log1p(cars['price'])

cars[['price','log_price']].describe().T


In [ ]:
# Feature engineering

# Reduce manufacturer to top 20, rest -> 'other'
if 'manufacturer' in cars.columns:
    top_man = cars['manufacturer'].value_counts().nlargest(20).index
    cars['manufacturer_simple'] = cars['manufacturer'].where(cars['manufacturer'].isin(top_man),'other')

# Age buckets
if 'age' in cars.columns:
    cars['age_bucket'] = pd.cut(cars['age'], bins=[-1,1,3,7,12,20,100], labels=['0-1','2-3','4-7','8-12','13-20','21+'])

# Odometer buckets
if 'odometer' in cars.columns:
    cars['odo_bucket'] = pd.cut(cars['odometer'], bins=[-1,5000,20000,50000,100000,200000,1e9],
                                 labels=['0-5k','5k-20k','20k-50k','50k-100k','100k-200k','200k+'])

cars[['manufacturer_simple','age_bucket','odo_bucket']].head()


In [ ]:
# Prepare dataset for modeling
from sklearn.model_selection import train_test_split

modeling_cols = ['log_price']
# pick a manageable set of predictors
predictors = []
for c in ['manufacturer_simple','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color','age_bucket','odo_bucket']:
    if c in cars.columns:
        predictors.append(c)

print('Predictors used:', predictors)

# Drop rows with NA in predictors
model_df = cars[['log_price'] + predictors].dropna()
print('modeling rows:', model_df.shape[0])

# One-hot encode categoricals (limit dummy columns by only taking top levels for high-cardinality)
# For columns with many categories, keep top 10 levels
def top_n_dummies(df, col, n=10):
    top = df[col].value_counts().nlargest(n).index
    return pd.get_dummies(df[col].where(df[col].isin(top),'other'), prefix=col, drop_first=True)

X_parts = []
for c in predictors:
    # treat categorical
    X_parts.append(top_n_dummies(model_df, c, n=10))

X = pd.concat(X_parts, axis=1)
y = model_df['log_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X shape:', X.shape)


In [ ]:
# Modeling: Linear Regression and RandomForest
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Metrics
import numpy as np

def metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {'rmse': rmse, 'r2': r2}

m_lr = metrics(y_test, y_pred_lr)
m_rf = metrics(y_test, y_pred_rf)

m_lr, m_rf


In [ ]:
# Feature importances (Random Forest) and Linear Regression coefficients

# RF importances
importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
importances.head(20)

# LR coefficients (show top positive/negative)
coef = pd.Series(lr.coef_, index=X.columns).sort_values(ascending=False)
pd.DataFrame({'coef': coef}).head(20)


In [ ]:
# Evaluate predictions on original price scale

# Convert from log back to price
y_test_price = np.expm1(y_test)
y_pred_rf_price = np.expm1(y_pred_rf)

# Print sample comparisons
pd.DataFrame({'actual_price': y_test_price, 'pred_price_rf': y_pred_rf_price}).head(10)

# Compute RMSE in dollars for RF
from sklearn.metrics import mean_squared_error
rmse_dollars = np.sqrt(mean_squared_error(y_test_price, y_pred_rf_price))
rmse_dollars


## 5. Evaluation & Recommendations

Based on the model results and feature importances:

- Highlight which features increase price most (manufacturer, condition, low mileage, newer age) and which lower price (high mileage, poor condition, salvage title).  
- Recommendation to dealership: prioritize acquiring vehicles from strong manufacturers and in good/excellent condition, and advertise low mileage and recent model years prominently.  
- Consider price segmentation strategies by age bucket and mileage bucket to optimize listing and pricing strategies.



In [ ]:
# Save a smaller cleaned sample for review
clean_sample_path = Path('/mnt/data/vehicles_clean_sample.csv')
model_df.sample(n=min(5000, model_df.shape[0]), random_state=42).to_csv(clean_sample_path, index=False)
print('Saved sample to', clean_sample_path)
